# Reinforcement Learning based Virtual Screening
Protein-Ligan pose prediction using RL

## Spatial block arrangement using RL CNN-DQN
__Input__
- Sandbox with block and the surface placemnt

__Output__
- <x, y, $\theta$> for block wrt Sandbox

In [ ]:
from block_world.env import Env
from agents.ddpg_agent import DDPGAgent
from matplotlib import pyplot as plt
import numpy as np

### Generating the sandbox
The block world generates a block and places it in the surface by randomizing <x, y, $\theta$>. The $\theta$ rotated block is stored in the _block_ property of the Block class.

Both the block and the surface are combined together into a single sandbox image. (More here)

In [ ]:
env = Env()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 20))
ax1.imshow(env.block.sandbox)
ax2.imshow(env.block.original_sandbox)
plt.show()

In [ ]:
block = env.block
env.block.update_sandbox(block.shift_x - block.block_x-1, block.shift_y - block.block_y -1, 0)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 20))
ax1.imshow(env.block.sandbox)
ax2.imshow(env.block.original_sandbox)
plt.show()

In [ ]:
print(env.block.distance(env.block.block_x, env.block.block_y, 0))

In [ ]:
xx = env.action_space.sample()
env.block.block_x
state, reward, t = env.step(xx)
plt.imshow(env.block.sandbox)
plt.show()
print(reward, xx, env.action_space.action_bounds.shape)

In [ ]:
block = env.block
print(env.block.block_x, env.block.block_y, block.shift_x, block.shift_y, [block.shift_x- block.block_x, block.shift_y - block.block_y, block.rotate_angle])

### Absolute Fit
The absolute fit is when the block is placed square on top of the slot.


  [block.shift_x- block.block_x, block.shift_y - block.block_y, block.rotate_angle]


In [ ]:
block = env.block
xx = [50, 50, -180]
state, reward, t = env.step(xx)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 20))
ax1.imshow(env.block.sandbox)
ax2.imshow(env.block.original_sandbox)
print(reward, xx, env.action_space.action_bounds.shape)

### Converting sandbox image to tensors for CNN consumption
The sandbox image needs to be converted into an image consumable by the CNN

In [ ]:
from block_world.env import Env
from agents.ddpg_agent import DDPGAgent
import numpy as np
env = Env()
agent = DDPGAgent(env)
actions =  agent.play(10000)

In [ ]:
env = agent.env
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 20))
env.block.update_sandbox()
ax1.imshow(env.block.sandbox)
ax2.imshow(env.block.original_sandbox)
plt.show()
print(env.block.block_x, env.block.block_y, env.block.shift_x, env.block.shift_y)